*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# lava-dnf 101: Overview of features

## Populations and connections

Create populations of leaky integrate-and-fire (LIF) neurons.
The `shape` argument determines the number of neurons (and their layout; see
further below).

In [18]:
from lava.proc.lif.process import LIF


# a one-dimensional LIF population
population = LIF(shape=(20,))

Create connections between populations using the `connect()` function.
The connectivity can be specified using a sequence of _Operations_. Here,
every neuron from `population1` is connected to the
corresponding neuron from `population2` with a synaptic weight of 20.
Operations are explained in more detail below.

In [19]:
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights


population1 = LIF(shape=(20,))
population2 = LIF(shape=(20,))
connect(population1.s_out, population2.a_in, ops=[Weights(20)])

## Dynamic neural fields (DNF)

### Multi-peak DNF

Create dynamic neural fields (DNFs) that support multiple peaks by using the
`MultiPeakKernel` with local excitation and mid-range inhibition. Use the
`Convolution` operation to apply the kernel.

In [20]:
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution


dnf = LIF(shape=(20,))

kernel = MultiPeakKernel(amp_exc=25,
                         width_exc=3,
                         amp_inh=-15,
                         width_inh=6)
connect(dnf.s_out, dnf.a_in, ops=[Convolution(kernel)])

### Selective DNF

Create DNFs that are selective and only create a single peak by using the
`SelectiveKernel` with local excitation and global inhibition.

In [21]:
from lava.lib.dnf.kernels.kernels import SelectiveKernel


dnf = LIF(shape=(20,))

kernel = SelectiveKernel(amp_exc=18,
                         width_exc=3,
                         global_inh=-15)
connect(dnf.s_out, dnf.a_in, ops=[Convolution(kernel)])

## Input

### Spike generators

To simulate spike input to a DNF, use a `RateCodeSpikeGen` Process. It
generates spikes with a spike rate pattern that can be specified, for
instance by using the `GaussPattern` Process. Connect the `RateCodeSpikeGen` to
 a DNF with the `connect()` function. You may change parameters of the
 `GaussPattern` during runtime.

In [22]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.rate_code_spike_gen.process import RateCodeSpikeGen


shape = (15,)

# GaussPattern produces a pattern of spike rates
gauss_pattern = GaussPattern(shape=shape, amplitude=100, mean=5, stddev=5)

# The spike generator produces spikes based on the spike rates given
# by the Gaussian pattern
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.a_out.connect(spike_generator.a_in)

# Connect the spike generator to a population
population = LIF(shape=shape)
connect(spike_generator.s_out, population.a_in, ops=[Weights(20)])

# Start running the network (explained below)
population.run(condition=RunSteps(num_steps=10),
               run_cfg=Loihi1SimCfg(select_tag='floating_pt'))

# You may change parameters of the Gaussian pattern during runtime
gauss_pattern.amplitude = 50

# Continue the run
# ...

# Stop the run to free resources
population.stop()

## Higher dimensions

Define DNFs and inputs over higher dimensionalities by specifying a `shape` with
multiple entries.

In [23]:
shape = (15, 15)
dnf = LIF(shape=shape)

Inputs and kernels must match the dimensionality of the DNF; specify
parameters that can be multi-dimensional, for example `mean` and `stddev` in
`GaussPattern`, as vectors rather than scalars.

In [24]:
# Make sure to specify 'mean' and 'stddev' as 2D vectors
gauss_pattern = GaussPattern(shape=shape,
                             amplitude=100,
                             mean=[5, 5],
                             stddev=[4, 4])
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.a_out.connect(spike_generator.a_in)

# Make sure to specify 'width_exc' and 'width_inh'
# as 2D vectors
kernel = MultiPeakKernel(amp_exc=58,
                         width_exc=[3.8, 3.8],
                         amp_inh=-50,
                         width_inh=[7.5, 7.5])
connect(dnf.s_out, dnf.a_in, ops=[Convolution(kernel)])

## Operations and larger architectures

### One-to-one connections
When connecting two DNFs that have the same shape (in terms of neurons and
dimensions), you can connect them without specifying any operations.

In [25]:
dnf1 = LIF(shape=(10,))
dnf2 = LIF(shape=(10,))

connect(dnf1.s_out, dnf2.a_in)

In that case the synaptic weight defaults to 1. If you want to set a
homogeneous weight for all neurons, use the operation `Weights`.
It connects each neuron in the first DNF to its (single) respective neuron
in the second DNF with the specified weight value.

In [26]:
dnf1 = LIF(shape=(10,))
dnf2 = LIF(shape=(10,))

connect(dnf1.s_out, dnf2.a_in, ops=[Weights(40)])

### Reducing dimensions
When the dimensionality of the source DNF is larger than that of
the target DNF, use the `ReduceDims` operation, specifying the indices of the
 dimensions that should be removed and how to remove them (here, by summing
 over dimension 1).

In [27]:
from lava.lib.dnf.operations.operations import ReduceDims
from lava.lib.dnf.operations.enums import ReduceMethod


dnf_2d = LIF(shape=(20, 10))
dnf_1d = LIF(shape=(20,))

connect(dnf_2d.s_out,
        dnf_1d.a_in,
        ops=[ReduceDims(reduce_dims=1, reduce_method=ReduceMethod.SUM)])

### Expanding dimensions
When the dimensionality of the source DNF is smaller than that of the target
DNF, use the `ExpandDims` operation, specifying the number of neurons of the
dimensions that will be added.

In [28]:
from lava.lib.dnf.operations.operations import ExpandDims


dnf_1d = LIF(shape=(20,))
dnf_2d = LIF(shape=(20, 10))

connect(dnf_1d.s_out, dnf_2d.a_in, ops=[ExpandDims(new_dims_shape=10)])

### Reordering dimensions
To reorder dimensions, use the `ReorderDims` operation, specifying the indices of the dimension in their new order.

In [29]:
from lava.lib.dnf.operations.operations import ReorderDims


dnf_1 = LIF(shape=(10, 20))
dnf_2 = LIF(shape=(20, 10))

# map dimensions (0, 1) of dnf_1 to dimensions (1, 0) of dnf_2
connect(dnf_1.s_out, dnf_2.a_in, ops=[ReorderDims(order=(1, 0))])

### Flipping dimensions
To flip dimensions, use the `Flip` operation, specifying the indices of the dimensions that should be flipped. The operation will map the first neuron of the input population to the last of the output population, the second to the second last, and so on.

In [30]:
from lava.lib.dnf.operations.operations import Flip

dnf_1 = LIF(shape=(10, 20))
dnf_2 = LIF(shape=(10, 20))

# connect the DNFs and flip the dimension of size 20
connect(dnf_1.s_out, dnf_2.a_in, ops=[Flip(dims=(1,))])

### Projecting along a diagonal
The operation `ReduceAlongDiagonal` sums the output of a higher-dimensional population of neurons along its main diagonal diagonal onto a lower-dimensional population. The operation `ExpandAlongDiagonal` does the inverse and projects a ridge of input from a lower-dimensional population into a higher-dimensional population along its main diagonal.
They both enable building relational networks (see the dedicated
[tutorial on relational networks](../relational_networks/tutorial_relational_networks.ipynb "Tutorial on relational networks")).

In [31]:
from lava.lib.dnf.operations.operations import ReduceAlongDiagonal

dnf_2d = LIF(shape=(40, 40))
relational_dnf_1d = LIF(shape=(79,))  # shape=(40 * 2 - 1,)

connect(dnf_2d.s_out, relational_dnf_1d.a_in, ops=[ReduceAlongDiagonal()])

In [32]:
from lava.lib.dnf.operations.operations import ExpandAlongDiagonal

relational_dnf_1d = LIF(shape=(79,))
dnf_2d = LIF(shape=(40, 40))  # shape=((79+1)/2, (79+1)/2)


connect(relational_dnf_1d.s_out, dnf_2d.a_in, ops=[ExpandAlongDiagonal()])

### Combining operations

All operations can be combined with each other to produce more complex
connectivity. For instance, reordering can be combined with the `ReduceDims` or
`ExpandDims` operation, which can again be combined with a `Weights` operation,
as shown below.

In [33]:
dnf_1d = LIF(shape=(10,))
dnf_2d = LIF(shape=(20, 10))

connect(dnf_1d.s_out, dnf_2d.a_in, ops=[ExpandDims(new_dims_shape=20),
                                        ReorderDims(order=(1, 0)),
                                        Weights(20)])

## Running and plotting networks

Call the `run()` method on any Process in the network.
To inspect Vars and Ports in Processes, create Monitors. Use them to probe Vars
and Ports before running. Create plots with the probed data after running.

In [34]:
from lava.proc.monitor.process import Monitor


shape = (15,)
time_steps = 10

# Set up a DNF
dnf = LIF(shape=shape)
kernel = MultiPeakKernel(amp_exc=17,
                         width_exc=3,
                         amp_inh=-15,
                         width_inh=6)
connect(dnf.s_out, dnf.a_in, ops=[Convolution(kernel)])

# Set up a monitor and probe the spike output of the DNF
monitor = Monitor()
monitor.probe(dnf.s_out, time_steps)

# Run the DNF
dnf.run(condition=RunSteps(num_steps=time_steps),
        run_cfg=Loihi1SimCfg(select_tag='floating_pt'))

# Get probed data from monitor
probed_data = monitor.get_data()

# Stop the execution after getting the monitor's data
dnf.stop()

# Now you can plot the data.